# Feature extraction and Data Visualization

ChatGPT's response when asked for traits that could be useful in identifying ai-generated articles.

1)  Average sentence length: AI-generated articles may have longer or shorter sentences than human-written articles on average.

2)  Punctuation usage: AI-generated articles may use certain types of punctuation, such as exclamation points or ellipses, more or less frequently than human-written articles.

3)  Word frequency: AI-generated articles may use certain words more or less frequently than human-written articles. For example, AI-generated articles may use more technical or jargon-y terms, or may use certain phrases or idioms less frequently.

4)  Part of speech tagging: By analyzing the parts of speech used in the article (such as nouns, verbs, and adjectives), we may be able to identify patterns that are more common in AI-generated articles.

5)  Named entity recognition: By analyzing the named entities (such as people, places, and organizations) mentioned in the article, we may be able to identify patterns that are more common in AI-generated articles.

6)  Syntactic complexity: By analyzing the complexity of sentence structures (such as the number of dependent clauses or subordinating conjunctions used), we may be able to identify patterns that are more common in AI-generated articles.

7)  Readability score: By calculating a readability score (such as the Flesch-Kincaid readability score), we may be able to identify patterns that are more common in AI-generated articles.


In [ ]:
import pandas as pd
import re
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import string
import scipy.stats as stats
import nltk

In [ ]:
df = pd.read_csv("final_dataset.csv")
df['Content'].replace('', np.nan, inplace=True)
df['Content'] = df.Content.apply(lambda x : x.strip())
df['Title'] = df.Title.apply(lambda x : x.strip())


#drop NA values
df = df.dropna()

# Select only specific columns from the dataset
selected_columns = ['Title', 'Content', 'Source']
df = df[selected_columns]


## Average Sentence Length

In [ ]:
def average_sentence_length(text):
    # Split text into sentences using regular expressions
    sentences = re.findall(r'\b[\w\s\',-]+\b[.?!]', text)
    # Calculate the average sentence length
    total_words = sum(len(sentence.split()) for sentence in sentences)
    num_sentences = len(sentences)
    if num_sentences > 0:
        return total_words / num_sentences
    else:
        return None

In [ ]:
df['AvgSentenceLength'] = df.Content.apply(lambda x : average_sentence_length(x))

In [ ]:
ai = df[df.Source == 1]
real = df[df.Source == 0]
real = real.dropna()


print(f'Mean sentence length for ai articles: {ai.AvgSentenceLength.mean()}, std: {ai.AvgSentenceLength.std()}')
print(f'Mean sentence length for real articles: {real.AvgSentenceLength.mean()}, std: {real.AvgSentenceLength.std()}')

In [ ]:
f, axes = plt.subplots(1, 2, figsize=(9,4))
axes[0].set_xlim(5, 45)
axes[1].set_xlim(5, 45)
sns.histplot(ai, x='AvgSentenceLength', color="orchid", ax=axes[0]).set(title='Average Sentence Length of AI articles')
sns.histplot(real, x='AvgSentenceLength', color="skyblue", ax=axes[1]).set(title='Average Sentence Length of Real articles')

In [ ]:
real = real.dropna()
stats.ttest_ind(ai.AvgSentenceLength, real.AvgSentenceLength)

In [ ]:
df = df.dropna()
stats.pearsonr(df.AvgSentenceLength, df.Source)

## Punctuation Usage

In [ ]:
def calculate_punctuation_percentage(text):
    # Remove all whitespace characters from the text
    text = "".join(text.split())

    # Calculate the length of the text and the length of the punctuation characters
    text_length = len(text)
    punctuation_length = len([c for c in text if c in string.punctuation])

    # Calculate the percentage of the text that is punctuation
    punctuation_percentage = (punctuation_length / text_length) * 100

    return punctuation_percentage

In [ ]:
df['PunctuationPercentage'] = df.Content.apply(lambda x : calculate_punctuation_percentage(x))

In [ ]:
ai = df[df.Source == 1]
real = df[df.Source == 0]


print(f'Mean punctuation percentage for ai articles: {ai.PunctuationPercentage.mean()}, std: {ai.PunctuationPercentage.std()}')
print(f'Mean punctuation percentage for real articles: {real.PunctuationPercentage.mean()}, std: {real.PunctuationPercentage.std()}')

In [ ]:
f, axes = plt.subplots(1, 2, figsize=(9,4))
axes[0].set_xlim(0, 9)
axes[1].set_xlim(0, 9)
sns.histplot(ai, x='PunctuationPercentage', color="orchid", ax=axes[0]).set(title='Punctuation Percentage of AI articles')
sns.histplot(real, x='PunctuationPercentage', color="skyblue", ax=axes[1]).set(title='Punctuation Percentage of Real articles')

In [ ]:
real = real.dropna()
stats.ttest_ind(ai.PunctuationPercentage, real.PunctuationPercentage)

In [ ]:
df = df.dropna()
stats.pearsonr(df.AvgSentenceLength, df.Source)

## Vocabulary Frequency

Uses a type-token ration, basically the number of unique words / total number of words

In [ ]:
def vocabulary_richness(text):
    # Tokenize the text into words
    words = nltk.word_tokenize(text)
    
    # Calculate the number of unique words (types)
    types = set(words)
    num_types = len(types)
    
    # Calculate the total number of words (tokens)
    num_tokens = len(words)
    
    # Calculate the type-token ratio
    if num_tokens > 0:
        ttr = num_types / num_tokens
    else:
        ttr = 0
        
    return ttr

In [ ]:
df['VocabRichness'] = df.Content.apply(lambda x : vocabulary_richness(x))

In [ ]:
ai = df[df.Source == 1]
real = df[df.Source == 0]


print(f'Mean punctuation percentage for ai articles: {ai.VocabRichness.mean()}, std: {ai.VocabRichness.std()}')
print(f'Mean punctuation percentage for real articles: {real.VocabRichness.mean()}, std: {real.VocabRichness.std()}')

In [ ]:
f, axes = plt.subplots(1, 2, figsize=(9,4))
# axes[0].set_xlim(0, 9)
# axes[1].set_xlim(0, 9)
sns.histplot(ai, x='VocabRichness', color="orchid", ax=axes[0]).set(title='Vocabulary Richness of AI articles')
sns.histplot(real, x='VocabRichness', color="skyblue", ax=axes[1]).set(title='Vocabulary Richness of Real articles')

In [ ]:
ai.sort_values(by='VocabRichness').tail(20)